In [11]:

import inspect
import os
from importlib import import_module

import numpy as np
import pandas as pd
from bilby.core.prior import LogUniform, PriorDict
from bilby.core.utils import logger
from bilby.hyper.model import Model
from gwpopulation.conversions import convert_to_beta_parameters
from gwpopulation.hyperpe import HyperparameterLikelihood, RateLikelihood
from gwpopulation.models.mass import (
    BrokenPowerLawPeakSmoothedMassDistribution,
    BrokenPowerLawSmoothedMassDistribution,
    MultiPeakSmoothedMassDistribution,
    SmoothedMassDistribution,
    two_component_primary_mass_ratio,
)
from gwpopulation.models.spin import (agn_spin, iid_spin, iid_spin_magnitude_beta,
                                      iid_spin_orientation_gaussian_isotropic,
                                      independent_spin_magnitude_beta,
                                      independent_spin_orientation_gaussian_isotropic)
from gwpopulation_pipe import vt_helper
from gwpopulation_pipe.parser import create_parser as create_main_parser
from gwpopulation_pipe.utils import prior_conversion


def create_parser():
    parser = create_main_parser()
    parser.add_argument("--prior", help="Prior file readable by bilby.")
    parser.add_argument(
        "--sampler_name",
        default="dynesty",
        help="Sampler to use, allowed options are pymultinest, dynesty, nestle, "
             "cpnest, emcee.",
    )
    parser.add_argument(
        "--models",
        type=str,
        action="append",
        help="Model functions to evaluate, default is "
             "two component mass and iid spins.",
    )
    parser.add_argument(
        "--vt-models",
        type=str,
        action="append",
        help="Model functions to evaluate for selection, default is no model",
    )
    parser.add(
        "--sampler-kwargs",
        type=str,
        default="Default",
        help=(
            "Dictionary of sampler-kwargs to pass in, e.g., {nlive: 1000} OR "
            "pass pre-defined set of sampler-kwargs {Default, FastTest}"
        ),
    )
    parser.add_argument(
        "--max-samples",
        default=1e10,
        type=int,
        help="Maximum number of posterior samples per event",
    )
    parser.add_argument(
        "--rate", default=False, type=bool, help="Whether to sample in the merger rate."
    )
    parser.add_argument(
        "--max-redshift", default=2.3, type=float, help="Maximum redshift in model."
    )
    return parser


def load_prior(args):
    hyper_prior = PriorDict(filename=args.prior)
    hyper_prior.conversion_function = prior_conversion
    if args.rate:
        hyper_prior["rate"] = LogUniform(
            minimum=1e-1,
            maximum=1e3,
            name="rate",
            latex_label="$R$",
            boundary="reflective",
        )
    return hyper_prior


MODEL_MAP = {
    "two_component_primary_mass_ratio": two_component_primary_mass_ratio,
    "iid_spin": iid_spin,
    "iid_spin_magnitude": iid_spin_magnitude_beta,
    "ind_spin_magnitude": independent_spin_magnitude_beta,
    "iid_spin_orientation": iid_spin_orientation_gaussian_isotropic,
    "two_comp_iid_spin_orientation": iid_spin_orientation_gaussian_isotropic,
    "ind_spin_orientation": independent_spin_orientation_gaussian_isotropic,
    "agn_spin_orientation": agn_spin,
    "SmoothedMassDistribution": SmoothedMassDistribution,
    "BrokenPowerLawSmoothedMassDistribution": BrokenPowerLawSmoothedMassDistribution,
    "MultiPeakSmoothedMassDistribution": MultiPeakSmoothedMassDistribution,
    "BrokenPowerLawPeakSmoothedMassDistribution": BrokenPowerLawPeakSmoothedMassDistribution,
}


def load_model(args):
    if args.models is None:
        args.models = [
            "two_component_primary_mass_ratio",
            "iid_spin",
            "gwpopulation.models.redshift.PowerLawRedshift",
        ]
    logger.info(f"Loading models = {args.models}")
    model = Model([_load_model(model, args) for model in args.models])
    return model


def load_vt(args):
    if args.vt_function == "" or args.vt_file == "None":
        return vt_helper.dummy_selection
    vt_model = Model([_load_model(model, args) for model in args.vt_models])
    try:
        vt_func = getattr(vt_helper, args.vt_function)
        return vt_func(args.vt_file, model=vt_model)
    except AttributeError:
        return vt_helper.injection_resampling_vt(vt_file=args.vt_file, model=vt_model)


def _load_model(model, args):
    if "." in model:
        split_model = model.split(".")
        module = ".".join(split_model[:-1])
        function = split_model[-1]
        _model = getattr(import_module(module), function)
        logger.info(f"Using {function} from {module}.")
    elif model in MODEL_MAP:
        _model = MODEL_MAP[model]
        logger.info(f"Using {model}.")
    else:
        raise ValueError(f"Model {model} not found.")
    if inspect.isclass(_model):
        if "redshift" in model.lower():
            kwargs = dict(z_max=args.max_redshift)
        else:
            kwargs = dict()
        _model = _model(**kwargs)
    return _model


def create_likelihood(args, posteriors, model, selection):
    if args.rate:
        likelihood_class = RateLikelihood
    else:
        likelihood_class = HyperparameterLikelihood
    likelihood = likelihood_class(
        posteriors,
        model,
        conversion_function=convert_to_beta_parameters,
        selection_function=selection,
        max_samples=args.max_samples,
    )

    return likelihood


def get_sampler_kwargs(args):
    if args.sampler_kwargs == "Default":
        sampler_kwargs = dict()
    elif not isinstance(args.sampler_kwargs, dict):
        sampler_kwargs = dict()
        for arg in args.sampler_kwargs:
            key = arg.split(":")[0].strip()
            value = arg.split(":")[1].strip()
            try:
                value = eval(value)
            except NameError:
                pass
            sampler_kwargs[key] = value
    else:
        sampler_kwargs = args.sampler_args
    if args.sampler_name == "cpnest" and "seed" not in sampler_kwargs:
        sampler_kwargs["seed"] = np.random.randint(0, 1e6)
    sampler_kwargs["nlive"] = 500
    sampler_kwargs["nact"] = 2
    sampler_kwargs["walks"] = 5
    return sampler_kwargs


def get_likelihood_and_hyoperprior_from_args(cli_str):
    parser = create_parser()
    cli_args = cli_str.split(" ")[1:]
    print(cli_args)
    args, unknown_args = parser.parse_known_args(cli_args)
    posterior_file = os.path.join(args.run_dir, "data", f"{args.data_label}.pkl")
    posteriors = pd.read_pickle(posterior_file)
    for ii, post in enumerate(posteriors):
        posteriors[ii] = post[post["redshift"] < args.max_redshift]
    vt_helper.N_EVENTS = len(posteriors)
    vt_helper.max_redshift = args.max_redshift
    logger.info(f"Loaded {len(posteriors)} posteriors")
    event_ids = list()
    with open(
            os.path.join(args.run_dir, "data",
                         f"{args.data_label}_posterior_files.txt"),
            "r",
    ) as ff:
        for line in ff.readlines():
            event_ids.append(line.split(":")[0])

    hyper_prior = load_prior(args)
    model = load_model(args)
    selection = load_vt(args)

    likelihood = create_likelihood(args, posteriors, model, selection)
    return likelihood, hyper_prior

In [16]:


LOCAL = "gwpopulation_pipe_analysis /Users/avaj0001/Documents/projects/agn_phenomenological_model/population_inference/test_outdir/test_config_complete.ini " \
        "--prior /Users/avaj0001/Documents/projects/agn_phenomenological_model/population_inference/priors/mass_c_iid_mag_agn_tilt_powerlaw_redshift.prior " \
        "--label test_mass_c_iid_mag_agn_tilt_powerlaw_redshift " \
        "--models SmoothedMassDistribution " \
        "--models iid_spin_magnitude " \
        "--models agn_spin_orientation " \
        "--models gwpopulation.models.redshift.PowerLawRedshift " \
        "--vt-models SmoothedMassDistribution " \
        "--vt-models gwpopulation.models.redshift.PowerLawRedshift"
CIT = "gwpopulation_pipe_analysis /home/avi.vajpeyi/projects/agn_phenomenological_model/population_inference/agn_outdir/agn_config_complete.ini --prior /home/avi.vajpeyi/projects/agn_phenomenological_model/population_inference/priors/mass_c_iid_mag_agn_tilt_powerlaw_redshift.prior --label agn_mass_c_iid_mag_agn_tilt_powerlaw_redshift --models SmoothedMassDistribution --models iid_spin_magnitude --models agn_spin_orientation --models gwpopulation.models.redshift.PowerLawRedshift --vt-models SmoothedMassDistribution --vt-models gwpopulation.models.redshift.PowerLawRedshift"

from pprint import pprint


    


def test_likelihood(likelihood, hyper_prior, sample):
    likelihood.parameters.update(theta)
    ln_l = likelihood.log_likelihood()
    ratio = likelihood.log_likelihood_ratio()
    params = {
        key: t for key, t in zip(hyper_prior.keys(), theta.values())
    }
    ln_p = hyper_prior.ln_prob(params)
    pprint(dict(ln_likelihood=ln_l, ln_prior=ln_p, log_likelihood_ratio=ratio))
    pprint(theta)




In [23]:
theta = {
    'alpha': 0.4505890454058221,
 'alpha_chi': 1.0,
 'amax': 1.0,
 'amax_1': 1,
 'amax_2': 1,
 'beta': 11.907712583354126,
 'beta_chi': 0.1,
 'delta_m': 5.817382019449902,
 'lam': 0.7706250922010177,
 'lamb': 0.0,
 'mmax': 37.597673822719635,
 'mmin': 7.245462219347805,
 'mpp': 31.0642830036645,
 'sigma_1': 1.0,
 'sigma_12': 0.3304983239337118,
 'sigpp': 2.840004026807676}
likelihood, hyperprior = get_likelihood_and_hyoperprior_from_args(LOCAL)
test_likelihood(likelihood, hyperprior, theta)

16:51 bilby INFO    : Loaded 2 posteriors
16:51 bilby INFO    : Loading models = ['SmoothedMassDistribution', 'iid_spin_magnitude', 'agn_spin_orientation', 'gwpopulation.models.redshift.PowerLawRedshift']
16:51 bilby INFO    : Using SmoothedMassDistribution.
16:51 bilby INFO    : Using iid_spin_magnitude.
16:51 bilby INFO    : Using agn_spin_orientation.
16:51 bilby INFO    : Using PowerLawRedshift from gwpopulation.models.redshift.
16:51 bilby INFO    : Using SmoothedMassDistribution.
16:51 bilby INFO    : Using PowerLawRedshift from gwpopulation.models.redshift.
16:51 bilby INFO    : Loading VT data from selection_effects/o1_o2_o3a_vt_data_aligned_low_spin.hdf5.


['/Users/avaj0001/Documents/projects/agn_phenomenological_model/population_inference/test_outdir/test_config_complete.ini', '--prior', '/Users/avaj0001/Documents/projects/agn_phenomenological_model/population_inference/priors/mass_c_iid_mag_agn_tilt_powerlaw_redshift.prior', '--label', 'test_mass_c_iid_mag_agn_tilt_powerlaw_redshift', '--models', 'SmoothedMassDistribution', '--models', 'iid_spin_magnitude', '--models', 'agn_spin_orientation', '--models', 'gwpopulation.models.redshift.PowerLawRedshift', '--vt-models', 'SmoothedMassDistribution', '--vt-models', 'gwpopulation.models.redshift.PowerLawRedshift']


16:51 bilby WARNING : Cannot import cupy, falling back to numpy.
/Users/avaj0001/Documents/projects/gwpopulation/gwpopulation/models/mass.py:462: RuntimeWarning: overflow encountered in exp
  window[smoothing_region] = 1 / (xp.exp(exponent) + 1)


{'ln_likelihood': nan,
 'ln_prior': -inf,
 'log_likelihood_ratio': -101.31003249203772}
{'alpha': 0.4505890454058221,
 'alpha_chi': 1.0,
 'amax': 1.0,
 'amax_1': 1,
 'amax_2': 1,
 'beta': 11.907712583354126,
 'beta_chi': 0.1,
 'delta_m': 5.817382019449902,
 'lam': 0.7706250922010177,
 'lamb': 0.0,
 'mmax': 37.597673822719635,
 'mmin': 7.245462219347805,
 'mpp': 31.0642830036645,
 'sigma_1': 1.0,
 'sigma_12': 0.3304983239337118,
 'sigpp': 2.840004026807676}


/Users/avaj0001/Documents/projects/gwpopulation/gwpopulation/models/mass.py:401: RuntimeWarning: invalid value encountered in true_divide
  p_q /= self.norm_p_q(beta=beta, mmin=mmin, delta_m=delta_m)
/Users/avaj0001/anaconda3/envs/phase-marginalisation-test/lib/python3.7/site-packages/bilby/core/prior/analytical.py:59: RuntimeWarning: invalid value encountered in multiply
  return np.nan_to_num(np.multiply(at_peak, np.inf))
/Users/avaj0001/anaconda3/envs/phase-marginalisation-test/lib/python3.7/site-packages/bilby/core/prior/base.py:157: RuntimeWarning: divide by zero encountered in log
  return np.log(self.prob(val))


In [22]:
import logging
logger.setLevel(logging.DEBUG)